As per the notebook `transformers-finetuned.ipynb`, we have finetuned a transformer model and saved it locally, as well as pushed it to the Huggingface Hub.

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

Retrieve our model from the Hub.

In [6]:
# Load the fine-tuned model (Unsloth likely returns a tuple, so unpack it)
model, metadata = FastLanguageModel.from_pretrained("slindgren/climate-mistral")
# Prepare the model for inference
FastLanguageModel.for_inference(model)
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("slindgren/climate-mistral")

==((====))==  Unsloth 2024.11.10: Fast Mistral patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA H100 PCIe. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [74]:
# Prompt

task = (
    "You are a text analysis machine. Respond only in a valid Python dictionary format. "
    "Use the following keys: 'topical-content', 'political-ideology-stance', and 'sentiment-tone'. "
    "All keys must have as their value full text descriptive sentences in the format of a python string. "
    "Do not include any additional text or comments. Only output the dictionary. "
    "Here is the text to analyze:\n"
)

# Input text
input_text = "Climate change is one of the most debated issues in modern politics. While some advocate for immediate and strong governmental intervention, others prioritize economic growth over environmental concerns. Public sentiment appears divided, with protests and activism on one side and climate skepticism on the other."

# Create a prompt to guide the model
prompt = (
    ""
    f"{task}"
    f"{input_text}"
)

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate the model's response
outputs = model.generate(
    **inputs, 
    max_length=2000, 
    temperature=0.2,  # Lower for more focused responses
    top_p=0.9,        # Narrow the sampling range
)


full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
output = full_output.replace(task,"")
output = output.replace(input_text,"")
output = output.replace("\n"," ")
output = output.replace("-","")


output

" {'topicalcontent': 'Climate change is a debated issue in modern politics. Some advocate for immediate and strong governmental intervention, while others prioritize economic growth over environmental concerns. Public sentiment is divided, with protests and activism on one side and climate skepticism on the other.',  'politicalideologystance': 'This text discusses the political debate surrounding climate change, with mentions of those advocating for governmental intervention and those prioritizing economic growth.',  'sentimenttone': 'The sentiment tone of this text is neutral, as it discusses the debate and division surrounding climate change without expressing a clear positive or negative opinion.'}"

In [ ]:
## PARSE THE OUTPUT INTO A REAL DICT